# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [59]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [60]:
dataset["Geography"].nunique()

3

In [61]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [62]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [63]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [64]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('cat', OneHotEncoder(), [1])], remainder='passthrough')
#column transformers are used to transform both categorial and numerical columns at a time
X = np.array(ct.fit_transform(X))

In [65]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [67]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [68]:
ann = tf.keras.models.Sequential()
#from tensorflow 2.0 ,tensorflow and keras are integrated.
#it is a sequence of layers starting from input layers and upto final output layers

### Adding the input layer and the first hidden layer

In [69]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
# in order to make a fully connected layer to model at any step,we use "Dense".
# add is a method in sequential class
# we first create a dense object and add this to ann

# units---no of neurons in hidden layer

### Adding the second hidden layer

In [70]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [71]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# output layer contains dimensions of output.simce here we have only one output for prediction
# the activation need to be changed from relu to sigmoid as sigmoid function allows to get only the probabilities
# and predictions of output

## Training the ANN

### Compiling the ANN

In [72]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#loss--loss function,metrics---method to evaluate
# as we are predicting the binary classification, we us only the 'binary_crossentropy'
# if we have three or more classifiction prediction,we use "categorical_cross_entropy" and then activation should be softmax

# optimiser--the optimiser(adam) is the best one for stochastic gradient descent
#optimiser updates the weights in order to reduce the loss error betwween predictions and new results

### Training the ANN on the Training set

In [82]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)
#batch_size is helpful in comparing several predictions(32) to same number of real results(32)
# value 32 is default value

Epoch 1/10
250/250 [==============================] - 0s 676us/step - loss: 0.3312 - accuracy: 0.8643
Epoch 2/10
250/250 [==============================] - 0s 661us/step - loss: 0.3312 - accuracy: 0.8641
Epoch 3/10
250/250 [==============================] - 0s 676us/step - loss: 0.3311 - accuracy: 0.8658
Epoch 4/10
250/250 [==============================] - 0s 679us/step - loss: 0.3313 - accuracy: 0.8646
Epoch 5/10
250/250 [==============================] - 0s 685us/step - loss: 0.3312 - accuracy: 0.8654
Epoch 6/10
250/250 [==============================] - 0s 667us/step - loss: 0.3309 - accuracy: 0.8651
Epoch 7/10
250/250 [==============================] - 0s 668us/step - loss: 0.3310 - accuracy: 0.8659
Epoch 8/10
250/250 [==============================] - 0s 737us/step - loss: 0.3308 - accuracy: 0.8651
Epoch 9/10
250/250 [==============================] - 0s 775us/step - loss: 0.3310 - accuracy: 0.8634
Epoch 10/10
250/250 [==============================] - ETA: 0s - loss: 0.3310 - ac

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**solution**

In [79]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# any prediction should be in 2d array
# we shold have the input in a scaled mannner
# as we sigmoid function,we get result in probabilities.so we use threshold probability as 0.5

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [83]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [84]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1508   87]
 [ 192  213]]


0.8605